In [144]:
!pip install polars beautifulsoup4 splinter selenium scikit-learn altair matplotlib tensorflow

In [145]:
#Data Manipulation and display toools
import polars as pl
from polars.exceptions import InvalidOperationError
import altair as alt
import numpy as np
import pathlib
# Pandas is necessary for ease of use with the ML libraries
# Polars does not use indexing and therefore does not the following data format properly
#   {tablename: {index1: [row1_values]},
#               {index2: [row2_values]}}
import pandas as pd
from IPython.display import display_html 
#Necessary for tensorflow on my machine due to distutils being depreciated
import setuptools

# Web scraping tools
import re
from bs4 import BeautifulSoup
# Uses firefox browser
from splinter import Browser
import time

# Data preprocessing tools
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Machine learning tools
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

random_state = 2112250415

# Loading Data

In [146]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Mild-Steel-Tempering':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")


Path is project root


In [147]:
resources_path = pathlib.PurePath(pathlib.PurePath(cwd), 'resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Mild-Steel-Tempering/resources')

### Many alloy composition columns were parsed incorrectly and failing to load

Several weight percent columns were parsed as int automatically due to having "0" for many initial rows. 

All weight percent columns should be parsed as float. 

In [148]:
data_path = f"{resources_path}/Raiipa-tempering-data.csv"
schema_overrides = {"C (%wt)" : pl.Float64,
"Mn (%wt)" : pl.Float64,
"P (%wt)" : pl.Float64,
"S (%wt)" : pl.Float64,
"Si (%wt)" : pl.Float64,
"Ni (%wt)" : pl.Float64,
"Cr (%wt)" : pl.Float64,
"Mo (%wt)" : pl.Float64,
"V (%wt)" : pl.Float64,
"Al (%wt)" : pl.Float64,
"Cu (%wt)" : pl.Float64}

df_data = pl.read_csv(data_path, schema_overrides=schema_overrides)
df_data.glimpse()

Rows: 1466
Columns: 17
$ Source                                  <str> 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956'
$ Steel type                              <str> 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026'
$ Initial hardness (HRC) - post quenching <str> '?', '?', '?', '?', '?', '?', '?', '?', '?', '?'
$ Tempering time (s)                      <i64> 600, 600, 600, 600, 600, 600, 600, 600, 600, 600
$ Tempering temperature (ºC)              <f64> 204.4, 260.0, 315.6, 371.1, 426.7, 482.2, 537.8, 593.3, 648.9, 704.4
$ C (%wt)                                 <f64> 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25
$ Mn 

In [149]:
# save initial columns names as they are very descriptive and may be useful later
initial_column_names = df_data.columns
initial_column_names

['Source',
 'Steel type',
 'Initial hardness (HRC) - post quenching',
 'Tempering time (s)',
 'Tempering temperature (ºC)',
 'C (%wt)',
 'Mn (%wt)',
 'P (%wt)',
 'S (%wt)',
 'Si (%wt)',
 'Ni (%wt)',
 'Cr (%wt)',
 'Mo (%wt)',
 'V (%wt)',
 'Al (%wt)',
 'Cu (%wt)',
 'Final hardness (HRC) - post tempering']

# Cleaning
Many columns need renaming for ease of manipulation

Columns need datatypes correction
 'Initial hardness (HRC) - post quenching' needs datatype correction. ? is NA value


In [150]:
dict_new_cnames = {}
for og_name in initial_column_names:
    # replace filler in hardness columns
    new_name = og_name.replace(' - ', '')
    
    # Handle units
    try:
        #if alloy weight percent remove units and return only elemental symbol
        if re.search(r"\(%wt\)", og_name):
            new_name = og_name.split(' ')[0]
            #skip the rest of the try block that will re_add the units to the end
            pass
        else: 
            #If not elemental composition, lowercase the string
            new_name = new_name.lower()
        # regex find the units inside the parenthesis, of the original name, not the lowercased new name
        # This lines breaks and goes to except if there is no units
        unit = re.search(r'\((\w+)\)', og_name).group(1)
        # replace the unit parenthesis string with parenthesis with an empty string
        new_name = re.sub(r"\(.+\)", "", new_name)
        # trim to whitespace end characters left by some unit removals
        new_name = new_name.rstrip()
        # append the unit string to the end of the processed name
        new_name = f"{new_name}_{unit}"
    except:
        # skip unit processing on names with no units denoted by parenthesis
        pass

    # Strip away special characters
    new_name = new_name.encode("ascii", errors="ignore").decode()
    #replace all whitespace with underscores
    new_name = new_name.replace(' ', '_')
    #add the name to the rename dict
    dict_new_cnames[og_name] = new_name
dict_new_cnames


{'Source': 'source',
 'Steel type': 'steel_type',
 'Initial hardness (HRC) - post quenching': 'initial_hardness_post_quenching_HRC',
 'Tempering time (s)': 'tempering_time_s',
 'Tempering temperature (ºC)': 'tempering_temperature_C',
 'C (%wt)': 'C',
 'Mn (%wt)': 'Mn',
 'P (%wt)': 'P',
 'S (%wt)': 'S',
 'Si (%wt)': 'Si',
 'Ni (%wt)': 'Ni',
 'Cr (%wt)': 'Cr',
 'Mo (%wt)': 'Mo',
 'V (%wt)': 'V',
 'Al (%wt)': 'Al',
 'Cu (%wt)': 'Cu',
 'Final hardness (HRC) - post tempering': 'final_hardness_post_tempering_HRC'}

In [151]:
df_clean_cnames = df_data.rename(dict_new_cnames)

## Clean data types and column values

In [152]:
count_of_qmark = df_clean_cnames['initial_hardness_post_quenching_HRC'].value_counts()\
    .filter(pl.col('initial_hardness_post_quenching_HRC') == "?")\
        .select("count").item()


In [153]:
percent_intial_hardness_unknown = count_of_qmark / int(df_clean_cnames['initial_hardness_post_quenching_HRC'].shape[0])
print(f"Unknown initial hardness: {round(percent_intial_hardness_unknown, 2) * 100:.0f}%")


Unknown initial hardness: 65%


In [154]:
df_clean_cnames.tail(3)


source,steel_type,initial_hardness_post_quenching_HRC,tempering_time_s,tempering_temperature_C,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC
str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Hollomon and Jaffe, 1945""","""1,15%C - plain carbon steel""","""64.5""",86400,500.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,32.0
"""Hollomon and Jaffe, 1945""","""1,15%C - plain carbon steel""","""64.5""",86400,600.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,23.0
"""Hollomon and Jaffe, 1945""","""1,15%C - plain carbon steel""","""64.5""",86400,700.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,4.5


In [155]:
df_clean_cnames["initial_hardness_post_quenching_HRC"].value_counts().sort('count', descending=True).head(3)

initial_hardness_post_quenching_HRC,count
str,u32
"""?""",949
"""66.5""",90
"""63.1""",51


In [156]:
df_clean = df_clean_cnames.with_columns(pl.col('initial_hardness_post_quenching_HRC').cast(pl.Float64, strict=False))


In [157]:
df_clean["initial_hardness_post_quenching_HRC"].value_counts().sort('count', descending=True).head(3)

initial_hardness_post_quenching_HRC,count
f64,u32
null,949
66.5,90
55.8,51


In [158]:
df_clean.describe()

statistic,source,steel_type,initial_hardness_post_quenching_HRC,tempering_time_s,tempering_temperature_C,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""1466""","""1466""",517.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0
"""null_count""","""0""","""0""",949.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,61.493617,21969.754434,422.024147,0.511583,0.74073,0.017236,0.023802,0.239379,0.362838,0.389696,0.080232,0.005457,0.034379,0.005986,41.468008
"""std""",null,null,5.656383,34177.623863,176.088041,0.224354,0.252913,0.007966,0.007967,0.239193,0.810091,0.480721,0.121422,0.02905,0.20534,0.019085,14.079248
"""min""","""Grange and Baughman, 1956""","""0,31%C - plain carbon steel""",46.5,10.0,100.0,0.25,0.3,0.007,0.005,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.9
"""25%""",null,null,58.8,600.0,260.0,0.37,0.6,0.012,0.018,0.16,0.0,0.02,0.0,0.0,0.0,0.0,32.1
"""50%""",null,null,63.1,3600.0,426.7,0.42,0.74,0.017,0.024,0.21,0.01,0.06,0.0,0.0,0.0,0.0,43.1
"""75%""",null,null,66.5,14400.0,593.3,0.56,0.8,0.019,0.029,0.24,0.06,0.8,0.22,0.0,0.0,0.0,51.8
"""max""","""Penha, 2010""","""Nitriding Steel """,67.0,115200.0,704.4,1.15,1.85,0.054,0.055,1.62,3.41,1.57,0.36,0.16,1.26,0.08,68.5


# Making steel types values searchable on AZoM.com
Steel types that do not include AISI and a code are not searchable in steel databased and need manual renaming

Exmaple: "0,74%C - plain carbon steel" is a AISI 1074 steel. Confirmed via elemental composition on https://www.azom.com/article.aspx?ArticleID=6558


In [159]:
steel_identifiers = ["steel_type", "source"]
df_clean[steel_identifiers].unique().shape

(36, 2)

## Renaming all possible steels so they are searchable

1945 source

I can not access source papers. Making a best guess of steel grade from elemental composition

I assume some alloying elements are a bit strange due to wartime shortages and potential contamination between alloys due to the push to increasew prodiction.

### additional Justifications are inline

In [160]:
#1945 source...
# I can not access source for the 1945 paper on these
# I assume they alloying elements are a bit strange due to wartime shortages
# and potentially contamination between alloys due to the push to increasew prodiction
dict_rename = {'0,98%C - plain carbon steel': 'AISI 1095', # copper and Cr, probably intentional?
                   '1,15%C - plain carbon steel': 'AISI 1095', # small chromium impurity?
                   '0,74%C - plain carbon steel': 'AISI 1074 Carbon Steel', 
                   '0,56%C - plain carbon steel': 'AISI 1055', # small chromium impurity?
                   '0,89%C - plain carbon steel': 'AISI 1090', # Cu discounted as impurity
                   'Nitriding Steel ': 'Non-searchable', 
                   '0,31%C - plain carbon steel': 'AISI 1030', # AISI 1030, Cr impurity
                   "AISI-SAE 9264" : 'AISI-SAE 9254', # access limited, may be AISI-SAE 9264 , not in AZoM
                   "AISI-SAE 2340" : 'Non-searchable', # access limited, may be SAE J2340, not in AZoM
                   "AISI-SAE 3140" : 'Non-searchable', # Not in AZoM elemental match to SAE 3140 https://www.steel-grades.com/metals/18/5155/-SAE-3140.html
                   "AISI-SAE 4068" : 'Non-searchable', # access limited, may be SAE 4068, not in AZoM
                   "AISI-SAE 4640" : "AISI 4640",
                   "AISI-SAE 4047" : "AISI 4047",
                   "AISI-SAE 1049" : "AISI 1049",
                   "AISI-SAE 6145" : "AISI 6145",
                   "AISI-SAE E52100" : "AISI 52100"} # # Not in AZoM elemental match to  SAE 4068https://www.steel-grades.com/Steel-Grades/Carbon-Steel/SAE-4068-.html

In [161]:
series_searchable_steel = df_clean.clone()
renamed = series_searchable_steel['steel_type'].replace(dict_rename)
series_searchable_steel = series_searchable_steel.with_columns(searchable = renamed)

In [162]:
series_to_search = series_searchable_steel['searchable'].unique()
steels_to_search = series_to_search.filter(series_to_search.eq('Non-searchable').not_())
steels_to_search

searchable
str
"""AISI-SAE 4340"""
"""AISI 1095"""
"""AISI-SAE 4140"""
"""AISI-SAE 1045"""
"""AISI-SAE 1042"""
…
"""AISI-SAE 1335"""
"""AISI-SAE 4027"""
"""AISI-SAE 1026"""


## Save Searchable steel
Checkpoint so that the code to this point does not need to be run every time

In [163]:
path_save_searchable = cwd / 'resources' / 'searchable_steels.csv'
pl.DataFrame(steels_to_search).write_csv(path_save_searchable)

# Web Scraping

Two Stages are necessary

AZoM does not have data on all steels.

MakeItFrom.com also does not have data from all steels. 

TODO: together they have data from ?? steels

In [164]:
len([print(steel) for steel in steels_to_search])

AISI-SAE 4340
AISI 1095
AISI-SAE 4140
AISI-SAE 1045
AISI-SAE 1042
AISI-SAE 1035
AISI-SAE 5160
AISI-SAE 9254
AISI-SAE 1030
AISI-SAE 1040
AISI-SAE 1065
AISI 1049
AISI 1055
AISI-SAE 4037
AISI 52100
AISI 1074 Carbon Steel
AISI 1030
AISI-SAE 1080
AISI-SAE 5140
AISI 1090
AISI-SAE 6150
AISI 4640
AISI 6145
AISI 4047
AISI-SAE 1335
AISI-SAE 4027
AISI-SAE 1026
AISI-SAE 1050
AISI-SAE 1038


29

## Scraping AZoM.com

In [165]:
browser = Browser('firefox')
base_url = "https://www.azom.com"
search_path = "/search.aspx?q="

In [166]:
# Search the steel 
def get_soup(soup):
    search_result = soup.find('div', class_='resultsContainer')
    first_item = search_result.find('a')
    # https://pytutorial.com/get-element-href-beautifulsoup/
    steel_link = first_item.get('href')
    time.sleep(0.15)
    browser.visit(f"{base_url}{steel_link}")
    soup2 = BeautifulSoup(browser.html, 'html.parser')
    return soup2


In [167]:
# already have elemental composition
# property - metric - imperial
# table_elements = tables[0]
# table_phys_props = tables[1]
# table_mecha_props = tables[2]
# table_therm_props = tables[3]
# table_other_desigs = tables[4]
#parse tables
def get_tables(soup):
    tables = []
    # parse all tables in the
    for html_table in soup.find_all('table'):
        table = []
        # parse rows in the table
        for tr in html_table.find_all('tr'):
            row = []
            #parse data cells in the row
            for t in tr.find_all(['th', 'td']):
                text = t.get_text(strip=True)
                #add data to row
                row.append(text)
            #add row to table
            table.append(row)
    #returns a list of list of lists
        tables.append(table)
    return tables
    

In [168]:
def make_dfs(lolols):
    tables = []
    # convert the list of lists to a dataframe
    for lol in lolols:
        try:
            df = pl.DataFrame(lol, orient='row')
            # rename columns using first row
            # https://stackoverflow.com/questions/75187317/how-to-rename-column-names-with-first-row-in-polars
            # [1:] removes the first row that replaced the column names 
            df = df.rename(df.head(1).to_dicts().pop())[1:]
            # add the dataframe to the list of tables on for this steel
            tables.append(df)
        except:
            tables.append(lol)
    return tables

In [169]:
dict_steel_tables = {}
for steel in steels_to_search:
    try:
        print("Working on:", steel)
        #search AZoM for the steel
        browser.visit(f"{base_url}{search_path}{steel}")
        soup_search = BeautifulSoup(browser.html, 'html.parser')
    
        #takes soup html and returns list of lists
        soup = get_soup(soup_search)
        lolols = get_tables(soup)
        dict_steel_tables[steel] = make_dfs(lolols)
        # tables list of lists and returns a dictionary
        # robots.txt indicates Crawl-delay: 120
        time.sleep(0.15)
    except: 
        dict_steel_tables[steel] = []

Working on: AISI-SAE 4340
Working on: AISI 1095
Working on: AISI-SAE 4140
Working on: AISI-SAE 1045
Working on: AISI-SAE 1042
Working on: AISI-SAE 1035
Working on: AISI-SAE 5160
Working on: AISI-SAE 9254
Working on: AISI-SAE 1030
Working on: AISI-SAE 1040
Working on: AISI-SAE 1065
Working on: AISI 1049
Working on: AISI 1055
Working on: AISI-SAE 4037
Working on: AISI 52100
Working on: AISI 1074 Carbon Steel
Working on: AISI 1030
Working on: AISI-SAE 1080
Working on: AISI-SAE 5140
Working on: AISI 1090
Working on: AISI-SAE 6150
Working on: AISI 4640
Working on: AISI 6145
Working on: AISI 4047
Working on: AISI-SAE 1335
Working on: AISI-SAE 4027
Working on: AISI-SAE 1026
Working on: AISI-SAE 1050
Working on: AISI-SAE 1038


In [170]:
dict_steel_tables

{'AISI-SAE 4340': [shape: (9, 2)
  ┌────────────────┬────────────────┐
  │ Element        ┆ Content (%)    │
  │ ---            ┆ ---            │
  │ str            ┆ str            │
  ╞════════════════╪════════════════╡
  │ Iron, Fe       ┆ 95.195 - 96.33 │
  │ Nickel, Ni     ┆ 1.65 - 2.00    │
  │ Chromium, Cr   ┆ 0.700 - 0.900  │
  │ Manganese, Mn  ┆ 0.600 - 0.800  │
  │ Carbon, C      ┆ 0.370 - 0.430  │
  │ Molybdenum, Mo ┆ 0.200 - 0.300  │
  │ Silicon, Si    ┆ 0.150 - 0.300  │
  │ Sulfur, S      ┆ 0.0400         │
  │ Phosphorous, P ┆ 0.0350         │
  └────────────────┴────────────────┘,
  shape: (2, 3)
  ┌───────────────┬────────────┬──────────────┐
  │ Properties    ┆ Metric     ┆ Imperial     │
  │ ---           ┆ ---        ┆ ---          │
  │ str           ┆ str        ┆ str          │
  ╞═══════════════╪════════════╪══════════════╡
  │ Density       ┆ 7.85 g/cm3 ┆ 0.284 lb/in³ │
  │ Melting point ┆ 1427°C     ┆ 2600°F       │
  └───────────────┴────────────┴────────────

# Converting AZoM Scraped Data to one dataframe

## Working with list of tables

Must confirma all tables were sucesffuly made into polars tables

Must reduce the numebr of tables to only those we are concerned with

* Remove "Other Designations" tables, including those that did not get sucessfully converted to a polars dataframe (table 4)

* Remove tables with properties that are not commonly present (thermal propterties table 3)

* Remove elemental compositions, already present in the data (table 0)

* Physical and mechanical properties are desired (tables 1,2)

In [171]:
dict_steel_tables_keep = {}
bool_check = True
polars_check = True 
for steel, tables in dict_steel_tables.items():
    all_polars = True
    count_polars = 0
    for table in tables:
        if isinstance(table, pl.DataFrame): 
            count_polars += 1 
    if count_polars != len(tables):
        all_polars = False
    if not (bool_check and all_polars):
        bool_check = False
    print("All polars?", all_polars, '|', len(tables), "Tables | Steel:", steel)
    dict_steel_tables_keep[steel] = tables[1:3]
print(polars_check)

All polars? False | 5 Tables | Steel: AISI-SAE 4340
All polars? True | 5 Tables | Steel: AISI 1095
All polars? True | 5 Tables | Steel: AISI-SAE 4140
All polars? True | 5 Tables | Steel: AISI-SAE 1045
All polars? True | 4 Tables | Steel: AISI-SAE 1042
All polars? False | 5 Tables | Steel: AISI-SAE 1035
All polars? True | 4 Tables | Steel: AISI-SAE 5160
All polars? True | 0 Tables | Steel: AISI-SAE 9254
All polars? True | 5 Tables | Steel: AISI-SAE 1030
All polars? True | 4 Tables | Steel: AISI-SAE 1040
All polars? True | 4 Tables | Steel: AISI-SAE 1065
All polars? True | 3 Tables | Steel: AISI 1049
All polars? True | 4 Tables | Steel: AISI 1055
All polars? True | 4 Tables | Steel: AISI-SAE 4037
All polars? True | 4 Tables | Steel: AISI 52100
All polars? True | 3 Tables | Steel: AISI 1074 Carbon Steel
All polars? True | 5 Tables | Steel: AISI 1030
All polars? True | 4 Tables | Steel: AISI-SAE 1080
All polars? True | 5 Tables | Steel: AISI-SAE 5140
All polars? True | 4 Tables | Steel: AI

### Check if all kept tables are good

Confirm all are polars dataframes

Look at the shape of all of them and determine which will be limiting

In [172]:
dict_phys_props = {}
dict_mech_props = {}

polars_check = True 
for steel, tables in dict_steel_tables_keep.items():
    try:
        all_polars = True
        count_polars = 0
        for table in tables:
            if isinstance(table, pl.DataFrame): 
                count_polars += 1 
        if count_polars != len(tables):
            all_polars = False
        if not (bool_check and all_polars):
            bool_check = False
        print("All polars?", all_polars, '|', len(tables), "Tables | Steel:", steel)

        phys_props = tables[0]
        mech_props = tables[1]
        print("Physical properties:", phys_props.shape, "| Properties head(1)", phys_props.select(pl.col('Properties').head(1)) )
        print("Mechanical propsshape:", mech_props.shape, "| Properties head(1)", mech_props.select(pl.col('Properties').head(1)) )
        dict_phys_props[steel] = phys_props
        dict_mech_props[steel] = mech_props
    except:
        dict_phys_props[steel] = pl.DataFrame({'Properties' : [], 'Metric' : []})
        dict_mech_props[steel] = pl.DataFrame({'Properties' : [], 'Metric' : []})

print("All Polars?", polars_check)

All polars? True | 2 Tables | Steel: AISI-SAE 4340
Physical properties: (2, 3) | Properties head(1) shape: (1, 1)
┌────────────┐
│ Properties │
│ ---        │
│ str        │
╞════════════╡
│ Density    │
└────────────┘
Mechanical propsshape: (14, 3) | Properties head(1) shape: (1, 1)
┌──────────────────┐
│ Properties       │
│ ---              │
│ str              │
╞══════════════════╡
│ Tensile strength │
└──────────────────┘
All polars? True | 2 Tables | Steel: AISI 1095
Physical properties: (2, 3) | Properties head(1) shape: (1, 1)
┌────────────┐
│ Properties │
│ ---        │
│ str        │
╞════════════╡
│ Density    │
└────────────┘
Mechanical propsshape: (14, 3) | Properties head(1) shape: (1, 1)
┌──────────────────┐
│ Properties       │
│ ---              │
│ str              │
╞══════════════════╡
│ Tensile strength │
└──────────────────┘
All polars? True | 2 Tables | Steel: AISI-SAE 4140
Physical properties: (2, 3) | Properties head(1) shape: (1, 1)
┌────────────┐
│ Propertie

# Filter for complete data, and select unit standard

Kpep only property and metric column from both sets of tables

Find which properties are present in all tables of each type

In [173]:
def remove_imperial(property_dict):
    new_dict = {}
    for key, table in property_dict.items():
        new_dict[key] = table.select(["Properties", "Metric"])
    return new_dict

dict_phys_props =  remove_imperial(dict_phys_props)
dict_mech_props = remove_imperial(dict_mech_props)


dict_phys_props['AISI-SAE 1050']

Properties,Metric
str,str
"""Density""","""7.85 g/cm3"""


# Properties to keep

Only properties present in all metals are the following:

Physical
* Density

Mechanical
* Elastic modulus
* Poisson's ratio

## Process Density into a df

In [174]:
#fix formatting in single cell. Did not incude a space 
# TODO: handle with regex to additional similar errors in additional rows
# arrives as '7.7-8.03g/cm3'
dict_phys_props['AISI 1074 Carbon Steel'][0, "Metric"] = '7.7-8.03 g/cm3'
# arrives as '200GPa'
dict_mech_props['AISI-SAE 1065'][2, "Metric"] = '200 GPa'
dict_mech_props['AISI-SAE 1065']

Properties,Metric
str,str
"""Tensile strength, ultimate""","""635 MPa"""
"""Tensile strength, yield""","""490 MPa"""
"""Modulus of elasticity""","""200 GPa"""
"""Bulk modulus (typical for stee…","""140 GPa"""
"""Shear modulus (typical for ste…","""80 GPa"""
…,…
"""Hardness, Knoop (converted fro…","""209"""
"""Hardness, Rockwell B (converte…","""90"""
"""Hardness, Rockwell C (converte…","""10"""


In [175]:
s_steel = pl.Series(name = 'steel', dtype= pl.String)
s_density = pl.Series(name = 'density', dtype= pl.String)
s_unit = pl.Series(name = 'units_density', dtype= pl.String)

for steel, table in dict_phys_props.items():
    try:
        #select only the  cell of interest
        df_density = table.filter(pl.col('Properties').str.contains("Density"))['Metric'].str.split(' ')
        density = df_density.list.get(0)
        unit = df_density.list.get(1)
        # Make a series of all 3
        s_steel.extend(pl.Series(name = 'steel', values = [steel]))
        s_density = s_density.extend(density)
        s_unit = s_unit.extend(unit)
    except:
        print("Failed on", steel)

Failed on AISI-SAE 9254


In [176]:
df_steel_properties = pl.DataFrame([s_steel, s_density, s_unit])
df_steel_properties

steel,density,units_density
str,str,str
"""AISI-SAE 4340""","""7.85""","""g/cm3"""
"""AISI 1095""","""7.85""","""g/cm3"""
"""AISI-SAE 4140""","""7.85""","""g/cm3"""
"""AISI-SAE 1045""","""7.87""","""g/cm3"""
"""AISI-SAE 1042""","""7.844""","""g/cm3"""
…,…,…
"""AISI-SAE 1335""","""7.87""","""g/cm3"""
"""AISI-SAE 4027""","""7.85""","""g/cm3"""
"""AISI-SAE 1026""","""7.858""","""g/"""


## Process mechanical properties

In [177]:
def grab_property(table, property):
    s_result = table.filter(pl.col('Properties')\
                            .str.to_lowercase()\
                                .str.contains(property))\
                                ['Metric']
    return s_result

In [178]:
def get_oobT(df, index):
    return df.list.get(index, null_on_oob=True)

In [179]:
# list of tuples if data was sucessfully scraped from AZoM
azom_sucessful = {'steel':[],'AZoM-Successful':[]}
list_of_scraped = [("steel", "pr", "em", "em_unit", "ys", "ys_unit", "uts", "uts_unit")]

for steel, table in dict_mech_props.items():
    pr = None
    em = None
    em_unit = None
    ys = None
    ys_unit = None
    uts = None
    uts_unit = None

    try:
        # Grab values and split into units where necessary
        #poisson's ratio
        pr = grab_property(table, "poisson")

        #elastic modulus
        df_em = grab_property(table, "elastic").str.split(' ')
        em = get_oobT(df_em, 0)
        em_unit = get_oobT(df_em, 1)
        # em = df_em.list.get(0, null_on_oob=True)
        # em_unit = df_em.list.get(1, null_on_oob=True)

        # grab yield strength
        df_ys = grab_property(table, "yield").str.split(' ')
        ys = get_oobT(df_ys, 0)
        ys_unit = get_oobT(df_ys, 1)

        # grab strength value that is not yield strength.
        # Tensile strength or ultimate yield strength or ultimate tensile strength. All the same measurement
        df_uts = table.filter(pl.col('Properties').str.to_lowercase().str.contains("strength") & 
                            pl.col('Properties').str.to_lowercase().str.contains("yield").not_())\
                                ['Metric'].str.split(' ')
        uts = get_oobT(df_uts, 0)
        uts_unit = get_oobT(df_uts, 1)
    
        # Make a tuple for each row
        # If this fails it will be an IndexError and go to the except
        row = (steel, pr[0], em[0], em_unit[0], ys[0], ys_unit[0], uts[0], uts_unit[0])

        list_of_scraped.append(row)
        azom_sucessful['steel'].append(steel)
        azom_sucessful['AZoM-Successful'].append(True)
        
    except (IndexError, InvalidOperationError):
        print("Failed on", steel)
        azom_sucessful['steel'].append(steel)
        azom_sucessful['AZoM-Successful'].append(False)

df_AZoM_successfull = pl.DataFrame(azom_sucessful)
df_AZoM_successfull.filter(pl.col('AZoM-Successful')==False)

Failed on AISI-SAE 1042
Failed on AISI-SAE 9254
Failed on AISI 1049
Failed on AISI-SAE 4037
Failed on AISI 52100
Failed on AISI-SAE 1080
Failed on AISI 4047
Failed on AISI-SAE 1335


steel,AZoM-Successful
str,bool
"""AISI-SAE 1042""",false
"""AISI-SAE 9254""",false
"""AISI 1049""",false
"""AISI-SAE 4037""",false
"""AISI 52100""",false
"""AISI-SAE 1080""",false
"""AISI 4047""",false
"""AISI-SAE 1335""",false


## Convert to dataframes and save

Checkpoint for continued work in this file so the scraping does not need to be done every time.

In [180]:
ttCdf_temp = pl.DataFrame(list_of_scraped, orient='row')
# rename columns using first row
# https://stackoverflow.com/questions/75187317/how-to-rename-column-names-with-first-row-in-polars
# [1:] removes the first row that replaced the column names 
ttCdf_temp = ttCdf_temp.rename(ttCdf_temp.head(1).to_dicts().pop())[1:]
ttCdf_temp.tail(3)

steel,pr,em,em_unit,ys,ys_unit,uts,uts_unit
str,str,str,str,str,str,str,str
"""AISI-SAE 1026""","""0.27-0.30""","""190-210""","""GPa""","""415""","""MPa""","""490""","""MPa"""
"""AISI-SAE 1050""","""0.27-0.30""","""190-210""","""GPa""","""580""","""MPa""","""690""","""MPa"""
"""AISI-SAE 1038""","""0.27-0.30""","""190-210""","""GPa""","""485""","""MPa""","""570""","""MPa"""


In [181]:
df_steel_properties = df_steel_properties.join(ttCdf_temp, on='steel', how='inner')
df_steel_properties.write_csv(f'{resources_path}/scraped_properties.csv')
df_steel_properties


steel,density,units_density,pr,em,em_unit,ys,ys_unit,uts,uts_unit
str,str,str,str,str,str,str,str,str,str
"""AISI-SAE 4340""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""470""","""MPa""","""745""","""MPa"""
"""AISI 1095""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""525""","""MPa""","""685""","""MPa"""
"""AISI-SAE 4140""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""415""","""MPa""","""655""","""MPa"""
"""AISI-SAE 1045""","""7.87""","""g/cm3""","""0.29""","""200""","""GPa""","""450""","""MPa""","""585""","""MPa"""
"""AISI-SAE 1035""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""370""","""MPa""","""585""","""MPa"""
…,…,…,…,…,…,…,…,…,…
"""AISI 6145""","""7.75""","""g/cm3""","""0.27-030""","""201-209""","""GPa""","""1165""","""MPa""","""1213""","""MPa"""
"""AISI-SAE 4027""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""325""","""MPa""","""515""","""MPa"""
"""AISI-SAE 1026""","""7.858""","""g/""","""0.27-0.30""","""190-210""","""GPa""","""415""","""MPa""","""490""","""MPa"""


In [182]:
s_AZOM_unsuccessful = df_AZoM_successfull.filter(pl.col('AZoM-Successful')==False)['steel']
pl.DataFrame(s_AZOM_unsuccessful).write_csv(f'{resources_path}/MIF_search.csv')

# Process units from scraped AZoM data

Ensure all units are base units and the same down each column

In [183]:
data_path = f"{resources_path}/scraped_properties.csv"
df_scraped = pl.read_csv(data_path)
df_scraped.glimpse()

Rows: 21
Columns: 10
$ steel         <str> 'AISI-SAE 4340', 'AISI 1095', 'AISI-SAE 4140', 'AISI-SAE 1045', 'AISI-SAE 1035', 'AISI-SAE 5160', 'AISI-SAE 1030', 'AISI-SAE 1040', 'AISI-SAE 1065', 'AISI 1055'
$ density       <str> '7.85', '7.85', '7.85', '7.87', '7.85', '7.85', '7.85', '7.845', '7.85', '7.85'
$ units_density <str> 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cc', 'g/cc', 'g/cm3', 'g/cm3'
$ pr            <str> '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.29', '0.27-0.30', '0.27 – 0.30', '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.27-0.30'
$ em            <str> '190-210', '190-210', '190-210', '200', '190-210', '190-210', '190-210', '190-210', '200', '190-210'
$ em_unit       <str> 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa'
$ ys            <i64> 470, 525, 415, 450, 370, 275, 440, 415, 490, 560
$ ys_unit       <str> 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa'
$ uts           <i64> 745, 685, 655, 585, 585, 724, 525, 620,

In [184]:
df_scraped.select(['units_density', 'em_unit', "ys_unit", "uts_unit"]).unique()

units_density,em_unit,ys_unit,uts_unit
str,str,str,str
"""g/cc""","""GPa""","""MPa""","""MPa"""
"""g/""","""GPa""","""MPa""","""MPa"""
"""g/cm3""","""GPa""","""MPa""","""MPa"""


## All units are valid and ready to be added to header and unit column removed

All units in units_density are a 1:1 conversion with g/cm3

g/mL (water) = g/cc = g/cm3 = grams per cubic centimeter

Converns:
* emdash and hyphens present
* AISI-SAE 1065 did not separate from it's decimal number.
* AISI-SAE 1026 indicatse g/ but was originally "g/ cm3"

In [185]:
df_scraped_unit_header = df_scraped.select(['steel', 
                                            'density', 
                                            'pr', 
                                            'em', 
                                            'ys',
                                            'uts']).rename({'density' : 'density_g_per_cm3',
                                                                        'em' : 'em_GPa',
                                                                        'ys' : 'ys_MPa',
                                                                        'uts' : 'uts_MPa'})

print(df_scraped_unit_header.shape)    
df_scraped_unit_header.head()

(21, 6)


steel,density_g_per_cm3,pr,em_GPa,ys_MPa,uts_MPa
str,str,str,str,i64,i64
"""AISI-SAE 4340""","""7.85""","""0.27-0.30""","""190-210""",470,745
"""AISI 1095""","""7.85""","""0.27-0.30""","""190-210""",525,685
"""AISI-SAE 4140""","""7.85""","""0.27-0.30""","""190-210""",415,655
"""AISI-SAE 1045""","""7.87""","""0.29""","""200""",450,585
"""AISI-SAE 1035""","""7.85""","""0.27-0.30""","""190-210""",370,585


In [186]:
df_scraped_unit_header.describe()

statistic,steel,density_g_per_cm3,pr,em_GPa,ys_MPa,uts_MPa
str,str,str,str,str,f64,f64
"""count""","""21""","""21""","""21""","""21""",21.0,21.0
"""null_count""","""0""","""0""","""0""","""0""",0.0,0.0
"""mean""",null,null,null,null,508.47619,680.190476
"""std""",null,null,null,null,223.223569,200.888183
"""min""","""AISI 1030""","""7.7-8.03""","""0.27 – 0.30""","""190-210""",275.0,490.0
"""25%""",null,null,null,null,415.0,570.0
"""50%""",null,null,null,null,450.0,650.0
"""75%""",null,null,null,null,525.0,690.0
"""max""","""AISI-SAE 6150""","""7.87""","""0.29""","""201-209""",1165.0,1276.0


## Process ranges to an average value

### Helper functions for mapping

In [187]:
df_scraped_clean = df_scraped_unit_header.clone()


In [188]:
# string to number conversion that is aware all values arefrom sklearn.cluster import KMeans, AgglomerativeClustering
def strip_string(string):
    return string.strip()

def if_no_decimal_point(num):
    #If leading 0 followed by number then replace leading 0 with "0."
    regex = r"^0\d"
    if re.search(regex, num):
        #replace the 
        num = re.sub("^0", "0.", num)
    return np.float64(num)

def process_ranged_val_cleaned(col, df=df_scraped_clean):
    # AISI-SAE 5160 and AISI-SAE 6150 use a different dash than the others for the pr range
    # Discovered when 2 nulls showed up after processing
        # mean could not process the string and resulted in a null
    s_same_dashes = df[col].str.replace('–', '-')
    # Split the range froma string into two strings of numbers
    s_split = s_same_dashes.str.split('-')
    # Convert these decimal points to numbers 
        #aware that some decimal values are formatted 0## with no .
    s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))
    # take middle of the two values
    return s_numeric_l.list.mean()

# Process Units 

In [189]:
# Split the range froma string into two strings of numbers
s_split = df_scraped_clean['density_g_per_cm3'].str.split('-')
# Convert these decimal points to numbers 
s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))
# take middle of the two values
s_mean_d = s_numeric_l.list.mean()

/tmp/ipykernel_24038/1347279945.py:4: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))


In [190]:
s_mean_p = process_ranged_val_cleaned("pr")
s_mean_em = process_ranged_val_cleaned("em_GPa")

/tmp/ipykernel_24038/2290863345.py:22: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))


In [191]:
df_AZoM_clean = df_scraped_clean.with_columns(poissons_ratio = s_mean_p, 
                                         density_g_per_cm3 = s_mean_d, 
                                         elastic_modulus_GPa = s_mean_em,
                                         yield_strength_MPa = df_scraped_unit_header['ys_MPa'],
                                         ultimate_strength_MPA = df_scraped_unit_header['uts_MPa'])\
                                            .select(["steel",
                                                     "density_g_per_cm3",
                                                     "elastic_modulus_GPa",
                                                     "yield_strength_MPa",
                                                     "ultimate_strength_MPA"])
df_AZoM_clean.describe()

statistic,steel,density_g_per_cm3,elastic_modulus_GPa,yield_strength_MPa,ultimate_strength_MPA
str,str,f64,f64,f64,f64
"""count""","""21""",21.0,21.0,21.0,21.0
"""null_count""","""0""",0.0,0.0,0.0,0.0
"""mean""",null,7.842048,200.47619,508.47619,680.190476
"""std""",null,0.031154,1.503963,223.223569,200.888183
"""min""","""AISI 1030""",7.75,200.0,275.0,490.0
"""25%""",null,7.85,200.0,415.0,570.0
"""50%""",null,7.85,200.0,450.0,650.0
"""75%""",null,7.85,200.0,525.0,690.0
"""max""","""AISI-SAE 6150""",7.87,205.0,1165.0,1276.0


# Drop columns that do not add information

Poisson's Ratio, Density, and Elastic Modulus are singletons. They do not add information.

In [192]:
df_AZoM_clean = df_AZoM_clean.select(['steel',
                                    "yield_strength_MPa",
                                    "ultimate_strength_MPA"])
df_AZoM_clean.describe()

statistic,steel,yield_strength_MPa,ultimate_strength_MPA
str,str,f64,f64
"""count""","""21""",21.0,21.0
"""null_count""","""0""",0.0,0.0
"""mean""",null,508.47619,680.190476
"""std""",null,223.223569,200.888183
"""min""","""AISI 1030""",275.0,490.0
"""25%""",null,415.0,570.0
"""50%""",null,450.0,650.0
"""75%""",null,525.0,690.0
"""max""","""AISI-SAE 6150""",1165.0,1276.0


In [193]:
df_AZoM_clean.write_csv(f'{resources_path}/scraped_properties.csv')

## Scraping MakeItFrom.com
DuckDuckGo search used since the steels of question are split in two categories and MakeItFrom does not have a search function.

In [194]:
# https://www.makeitfrom.com/material-group/Wrought-Alloy-Steel-SAE-AISI
# OR
# https://www.makeitfrom.com/material-group/Wrought-Carbon-Or-Non-Alloy-Steel
base_url = "https://www.makeitfrom.com"
search_url = "https://duckduckgo.com/?hps=1&q="
search_suffix = "+site%3Amakeitfrom.com&atb=v427-1&ia=web"

In [195]:
s_AZOM_unsuccessful

steel
str
"""AISI-SAE 1042"""
"""AISI-SAE 9254"""
"""AISI 1049"""
"""AISI-SAE 4037"""
"""AISI 52100"""
"""AISI-SAE 1080"""
"""AISI 4047"""
"""AISI-SAE 1335"""


In [196]:
def handle_ranges(values_list):
    if values_list[1] == 'to':
        # map the range of values to a list of integers, then take the mean of it.
        value = np.mean(list(map(int, values_list[0:3:2])))
        units = values_list[3]
    else:
        value = np.float64(values_list[0])
        units = values_list[1]
    return value, units

In [197]:
def get_row(soup):
    mech_props = soup.find_all('div', class_='mech')
    # -2 is a magic number
    # the last 2 values are always tensile strength
    uts = 'no_label'
    uts_units = 'no_units'
    ys = 'no_label'
    ys_units = 'no_units'

    for div in mech_props[-2:]:
        ps = div.find_all('p')
        # get the property name
        label = ps[0].text
        # get the string of values and units and split it
        values = ps[1].text.split(' ')
        # Grab the metric value 
        value, units = handle_ranges(values)
        if 'uts' in label.lower():
            uts = value
            uts_units = units
        elif 'yield' in label.lower():
            ys = value
            ys_units = units
    row = (steel, ys, ys_units, uts, uts_units)
    return row

In [198]:
def handle_cold_drawn():
    # Browser is already on steel page
    soup = BeautifulSoup(browser.html, 'html.parser')
    cold_drawn_present = True
    steel_page = soup.find('div', class_='split links break-mid')
    # https://stackoverflow.com/questions/33404049/navigation-with-beautifulsoup
    
    cold_drawn = steel_page.find('a', string= lambda text: text \
                            #cold drawn matches values from AZoM for steels that are on both
                                and "Cold Drawn" in text\
                                # not "and" removed other processing in addtional to cold drawn if applicable
                                and not "and" in text) 
    try: 
        cold_drawn_page = cold_drawn.get('href')
        time.sleep(0.15)
        browser.visit(f'{base_url}{cold_drawn_page}')
    except AttributeError:
        # do not go to the cold drawn page if not present
        pass    

In [199]:

list_of_scraped = [("steel", "ys", "ys_unit", "uts", "uts_unit")] 
for steel in s_AZOM_unsuccessful:
    print("Working on:", steel)
    #search MIF for the steel
    time.sleep(0.15)
    browser.visit(f"{search_url}{steel}{search_suffix}")
    soup_search = BeautifulSoup(browser.html, 'html.parser')
    steel_link = soup_search.find('a',{'data-testid': "result-extras-url-link"}).get('href')
    
    time.sleep(0.15)
    browser.visit(steel_link)
    # handle cold drawn will land us on the desired steel page
    handle_cold_drawn()
    soup_results = BeautifulSoup(browser.html, "html.parser")
    list_of_scraped.append(get_row(soup_results))

browser.quit()

list_of_scraped

Working on: AISI-SAE 1042
Working on: AISI-SAE 9254
Working on: AISI 1049
Working on: AISI-SAE 4037
Working on: AISI 52100
Working on: AISI-SAE 1080
Working on: AISI 4047
Working on: AISI-SAE 1335


[('steel', 'ys', 'ys_unit', 'uts', 'uts_unit'),
 ('AISI-SAE 1042', np.float64(580.0), 'MPa', np.float64(700.0), 'MPa'),
 ('AISI-SAE 9254', np.float64(410.0), 'MPa', np.float64(660.0), 'MPa'),
 ('AISI 1049', np.float64(640.0), 'MPa', np.float64(750.0), 'MPa'),
 ('AISI-SAE 4037', np.float64(290.0), 'MPa', np.float64(540.0), 'MPa'),
 ('AISI 52100', np.float64(460.0), 'MPa', np.float64(1300.0), 'MPa'),
 ('AISI-SAE 1080', np.float64(535.0), 'MPa', np.float64(820.0), 'MPa'),
 ('AISI 4047', np.float64(310.0), 'MPa', np.float64(580.0), 'MPa'),
 ('AISI-SAE 1335', np.float64(300.0), 'MPa', np.float64(550.0), 'MPa')]

In [200]:
df_MIF = pl.DataFrame(list_of_scraped, orient='row')
df_MIF = df_MIF.rename(df_MIF.head(1).to_dicts().pop())[1:]
df_MIF

steel,ys,ys_unit,uts,uts_unit
str,str,str,str,str
"""AISI-SAE 1042""","""580""","""MPa""","""700""","""MPa"""
"""AISI-SAE 9254""","""410""","""MPa""","""660""","""MPa"""
"""AISI 1049""","""640""","""MPa""","""750""","""MPa"""
"""AISI-SAE 4037""","""290""","""MPa""","""540""","""MPa"""
"""AISI 52100""","""460""","""MPa""","""1300""","""MPa"""
"""AISI-SAE 1080""","""535""","""MPa""","""820""","""MPa"""
"""AISI 4047""","""310""","""MPa""","""580""","""MPa"""
"""AISI-SAE 1335""","""300""","""MPa""","""550""","""MPa"""


In [201]:
df_MIF_clean = df_MIF.select(['steel', 'ys', 'uts']).rename({'ys' : 'yield_strength_MPa',
                                                             'uts' : 'ultimate_strength_MPA'})
df_MIF_clean

steel,yield_strength_MPa,ultimate_strength_MPA
str,str,str
"""AISI-SAE 1042""","""580""","""700"""
"""AISI-SAE 9254""","""410""","""660"""
"""AISI 1049""","""640""","""750"""
"""AISI-SAE 4037""","""290""","""540"""
"""AISI 52100""","""460""","""1300"""
"""AISI-SAE 1080""","""535""","""820"""
"""AISI 4047""","""310""","""580"""
"""AISI-SAE 1335""","""300""","""550"""


In [202]:
df_AZoM_clean

steel,yield_strength_MPa,ultimate_strength_MPA
str,i64,i64
"""AISI-SAE 4340""",470,745
"""AISI 1095""",525,685
"""AISI-SAE 4140""",415,655
"""AISI-SAE 1045""",450,585
"""AISI-SAE 1035""",370,585
…,…,…
"""AISI 6145""",1165,1213
"""AISI-SAE 4027""",325,515
"""AISI-SAE 1026""",415,490


In [203]:
df_complete_props = pl.concat([df_AZoM_clean, df_MIF_clean], how='vertical_relaxed')
df_complete_props = df_complete_props.with_columns(pl.col('yield_strength_MPa').cast(pl.Int64),
                                       pl.col('ultimate_strength_MPA').cast(pl.Int64))
df_complete_props

steel,yield_strength_MPa,ultimate_strength_MPA
str,i64,i64
"""AISI-SAE 4340""",470,745
"""AISI 1095""",525,685
"""AISI-SAE 4140""",415,655
"""AISI-SAE 1045""",450,585
"""AISI-SAE 1035""",370,585
…,…,…
"""AISI-SAE 4037""",290,540
"""AISI 52100""",460,1300
"""AISI-SAE 1080""",535,820


In [204]:
df_complete_props.write_csv(f'{resources_path}/scraped_properties.csv')

# Join properties data with tempering data
Inner join to exclude the steels that are not searchable with the given information, or that do not conform to any known and accessable standards.

1286 rows after join

In [205]:
df_complete_props = df_complete_props.rename({'steel':'searchable'})
data = series_searchable_steel.join(df_complete_props, how="inner", on='searchable')

In [206]:
data.shape

(1286, 20)

# Select columns that have relevant data for the analysis

Exclude:
* Source data - does not provide information about the problem
* initial hardness - very incomplete data and no accurate way to fill it
* searchable -  contains more information than searchable, but is likely relipcated in elemental
  * Searchable contains fewer values, this grouping was used to scrape the strength values and contains no additional information
  * steel_type contains steel denotation from the papers. The informationa in this column is replicated in the elemental composition columns
  * PCA will handle this


In [207]:
df_relevent = data.drop(['source', 'searchable', 'initial_hardness_post_quenching_HRC'])
df_relevent.columns

['steel_type',
 'tempering_time_s',
 'tempering_temperature_C',
 'C',
 'Mn',
 'P',
 'S',
 'Si',
 'Ni',
 'Cr',
 'Mo',
 'V',
 'Al',
 'Cu',
 'final_hardness_post_tempering_HRC',
 'yield_strength_MPa',
 'ultimate_strength_MPA']

# Preprocessing the data for machine learning

In [208]:
target_columns = [ 'tempering_time_s', 'tempering_temperature_C']

X_prepre = df_relevent.drop(target_columns)

y_prepre = df_relevent.select(target_columns)

In [209]:
y_prepre.head(3)

tempering_time_s,tempering_temperature_C
i64,f64
600,204.4
600,260.0
600,315.6


# Preprocess target values

All are converted to Celcius but several are very near together because theyw ere originally Farenheight.

eg 200 C is negligably near 204.4 C which equals 400 F.

55 C ranges were picked, as it makes 11 whole number bins from 100 to 705


## Bin tempering values by 55 C

38 C was considered (100 F) but some values were indistinct. 
55 C is a whole number value that results in a whole number of bins.

It results in 11 bins. 

This is the same number of bins when we bin tempering time <= 5 minutes

In [210]:
tempering_bins = range(155, 705, 55)
list(tempering_bins)

[155, 210, 265, 320, 375, 430, 485, 540, 595, 650]

In [211]:
labels = ['100', '200', '250', '300', '350', '400', '450', '500', '550', '600', '700']

In [212]:
#binned_temperting_tempC = 
tempering_tempC_binned = y_prepre['tempering_temperature_C'].cut(tempering_bins, labels=labels)
tempering_tempC_binned


tempering_temperature_C
cat
"""200"""
"""250"""
"""300"""
"""350"""
"""400"""
…
"""300"""
"""400"""
"""500"""


# Bin Tempering time (s)

Short tempering times may matter at very high temperature, but they were supported by too few samples.

bin all values less or equal to 5 minutes together (300 s)

In [213]:
# y_prepre['tempering_time_s'].value_counts().filter(pl.col('tempering_time_s').le(300))

In [214]:
# tempering_times_to_replace = y_prepre['tempering_time_s'].le(300)
# tempering_times_to_replace


In [215]:
# https://stackoverflow.com/questions/75211934/how-can-i-use-when-then-and-otherwise-with-multiple-conditions-in-polars
# Source for how to amake this replacement
# When column values less than 300 replace with 300, else keep value
# tts_bin_le_5min = y_prepre.with_columns(pl.when(pl.col('tempering_time_s').le(300))\
#                                 .then(5)\
#                                     .otherwise(pl.col('tempering_time_s') / 60)\
#                                         .alias('tts_bin_le_5min'))

# tts_bin_string = tts_bin_le_5min.with_columns(pl.col('tts_bin_le_5min').cast(pl.String)\
#                             .replace('5.0', '<= 5'))\
#                             .select('tts_bin_le_5min')

In [216]:
# tts_bin_string.unique()


In [217]:
y = pl.DataFrame({'tempering_time_s_proc': y_prepre['tempering_time_s'],
    #'tempering_time_s_proc': tts_bin_string,
                   'tempering_temperature_C_proc': tempering_tempC_binned})
y

tempering_time_s_proc,tempering_temperature_C_proc
i64,cat
600,"""200"""
600,"""250"""
600,"""300"""
600,"""350"""
600,"""400"""
…,…
86400,"""300"""
86400,"""400"""
86400,"""500"""


In [218]:
y.write_csv(f"{resources_path}/y.csv")

# Preprocess data values

In [219]:
print(X_prepre.shape)
X_prepre.head(3)

(1286, 15)


steel_type,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC,yield_strength_MPa,ultimate_strength_MPA
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
"""AISI-SAE 1026""",0.25,0.79,0.012,0.026,0.11,0.0,0.0,0.0,0.0,0.0,0.0,50.6,415,490
"""AISI-SAE 1026""",0.25,0.79,0.012,0.026,0.11,0.0,0.0,0.0,0.0,0.0,0.0,48.3,415,490
"""AISI-SAE 1026""",0.25,0.79,0.012,0.026,0.11,0.0,0.0,0.0,0.0,0.0,0.0,43.7,415,490


# Encode and scale variables
Most columns are numeric and range from 0.0# to 1000 so scaling is necessary
steel names are categorical and would benefit from one-hot encoding.

## One hot encode steel types

Encoding chosen because this is a categorical for a given material and testing suite

In [220]:
to_scale = X_prepre.drop('steel_type')
to_onehot = X_prepre.select('steel_type')

In [221]:
to_onehot

steel_type
str
"""AISI-SAE 1026"""
"""AISI-SAE 1026"""
"""AISI-SAE 1026"""
"""AISI-SAE 1026"""
"""AISI-SAE 1026"""
…
"""1,15%C - plain carbon steel"""
"""1,15%C - plain carbon steel"""
"""1,15%C - plain carbon steel"""


In [248]:
# create the transformers
ohe = OneHotEncoder(handle_unknown='ignore')

oh_transformed = ohe.fit_transform(to_onehot)

# get the transformed column names
ohe_names = ohe.get_feature_names_out()

# 4 stack exchange methods tried. Then I made this one up ¯\_(ツ)_/¯
# Combinated of recenly changed API and polars I think, no way to rename all columns like in pandas.
columns = [f'column_{x}' for x in range(0, 30)]
rename_dict = dict(zip(columns, ohe_names))

df_ohe_encoded = pl.DataFrame(oh_transformed.toarray()).rename(rename_dict)
df_ohe_encoded.tail(3)

"steel_type_0,31%C - plain carbon steel","steel_type_0,56%C - plain carbon steel","steel_type_0,74%C - plain carbon steel","steel_type_0,89%C - plain carbon steel","steel_type_0,98%C - plain carbon steel","steel_type_1,15%C - plain carbon steel",steel_type_AISI-SAE 1026,steel_type_AISI-SAE 1030,steel_type_AISI-SAE 1035,steel_type_AISI-SAE 1038,steel_type_AISI-SAE 1040,steel_type_AISI-SAE 1042,steel_type_AISI-SAE 1045,steel_type_AISI-SAE 1049,steel_type_AISI-SAE 1050,steel_type_AISI-SAE 1065,steel_type_AISI-SAE 1080,steel_type_AISI-SAE 1335,steel_type_AISI-SAE 4027,steel_type_AISI-SAE 4037,steel_type_AISI-SAE 4047,steel_type_AISI-SAE 4140,steel_type_AISI-SAE 4340,steel_type_AISI-SAE 4640,steel_type_AISI-SAE 5140,steel_type_AISI-SAE 5160,steel_type_AISI-SAE 6145,steel_type_AISI-SAE 6150,steel_type_AISI-SAE 9264,steel_type_AISI-SAE E52100
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Scale numerical columns 
This will ensure the small elemental composition values are not overshadowed by the large strength properties 

In [249]:
print(type(to_scale))
to_scale.tail(3)

<class 'polars.dataframe.frame.DataFrame'>


C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC,yield_strength_MPa,ultimate_strength_MPA
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,32.0,525,685
1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,23.0,525,685
1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,4.5,525,685


In [250]:
scaler = StandardScaler()

X_scaler = scaler.fit(to_scale)

scaled = scaler.transform(to_scale)

scaler_names = scaler.get_feature_names_out()
columns = [f'column_{x}' for x in range(0, 14)]
rename_dict = dict(zip(columns, scaler_names))

df_scaled = pl.DataFrame(scaled).rename(rename_dict)
df_scaled.tail(3)


C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC,yield_strength_MPa,ultimate_strength_MPA
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,-0.682819,0.188703,-0.105424
2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,-1.309217,0.188703,-0.105424
2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,-2.596813,0.188703,-0.105424


## Merge one hot encoded variables with scaled numeric variables

In [251]:
X = pl.concat([df_ohe_encoded, df_scaled], how='horizontal')
X

"steel_type_0,31%C - plain carbon steel","steel_type_0,56%C - plain carbon steel","steel_type_0,74%C - plain carbon steel","steel_type_0,89%C - plain carbon steel","steel_type_0,98%C - plain carbon steel","steel_type_1,15%C - plain carbon steel",steel_type_AISI-SAE 1026,steel_type_AISI-SAE 1030,steel_type_AISI-SAE 1035,steel_type_AISI-SAE 1038,steel_type_AISI-SAE 1040,steel_type_AISI-SAE 1042,steel_type_AISI-SAE 1045,steel_type_AISI-SAE 1049,steel_type_AISI-SAE 1050,steel_type_AISI-SAE 1065,steel_type_AISI-SAE 1080,steel_type_AISI-SAE 1335,steel_type_AISI-SAE 4027,steel_type_AISI-SAE 4037,steel_type_AISI-SAE 4047,steel_type_AISI-SAE 4140,steel_type_AISI-SAE 4340,steel_type_AISI-SAE 4640,steel_type_AISI-SAE 5140,steel_type_AISI-SAE 5160,steel_type_AISI-SAE 6145,steel_type_AISI-SAE 6150,steel_type_AISI-SAE 9264,steel_type_AISI-SAE E52100,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC,yield_strength_MPa,ultimate_strength_MPA
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,0.611736,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,0.451657,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,0.131498,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,-0.091221,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,-0.31394,-0.342255,-1.009596
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,0.604776,0.188703,-0.105424
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,0.013178,0.188703,-0.105424
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,-0.682819,0.188703,-0.105424


In [252]:
X.write_csv(f"{resources_path}/X.csv")

# Modelling

## PCA analysis

Introduction of two strength metrics certainly introduced multcolinearity, with eachother and with the elemental composision, some of whicha re indicators of strength.

PCA analysis will be done to alleiviate these factors.

In [253]:
X = pl.read_csv(f"{resources_path}/X.csv")

In [254]:
pca_explained_sum = {'x': [], 'pca_explained_var': []}
pca_explained_min = {'x': [], 'pca_explained_var': []}
for x in range(3,30):
    pca = PCA(n_components=x ,random_state=random_state)

    pca.fit(X)

    pca_explained_sum['x'].append(x)
    sum_explained = np.sum(pca.explained_variance_ratio_)
    pca_explained_sum['pca_explained_var'].append(sum_explained)

    pca_explained_min['x'].append(x)
    min_explained = np.min(pca.explained_variance_ratio_)
    pca_explained_min['pca_explained_var'].append(min_explained)

In [255]:
df_pca_explained = pl.DataFrame(pca_explained_sum)
df_pca_explained

x,pca_explained_var
i64,f64
3,0.502728
4,0.610417
5,0.6986
6,0.761974
7,0.817106
…,…
25,0.993986
26,0.996071
27,0.99727


## PCA component number determination

In [256]:
alt.Chart(df_pca_explained,
          title= "Total Explained Vairance vs Nubmer of PCA Features").mark_line().encode(
    alt.X('x:Q').title("PCA X"),
    alt.Y('pca_explained_var:Q')\
        .scale(domain=(0.5,1))\
            .title("Total explained variance")
)

alt.Chart(...)

### Value of increasing PCA components dramatically decreases between 10 and 15

In [257]:
df_pca_explained_min = pl.DataFrame(pca_explained_min)
df_pca_explained_min.filter(pl.col('x').le(15) & pl.col('x').ge(10))

x,pca_explained_var
i64,f64
10,0.026609
11,0.01642
12,0.012011
13,0.007145
14,0.003059
15,0.002843


### PCA component number 12 selected
Component 13 has half the value as component 12

In [258]:
pca = PCA(n_components= 12, random_state=random_state)

pca.fit(X)

X_pca = pca.transform(X)

df_pca = pl.DataFrame(X_pca)
df_pca.head(3)

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-1.321473,0.430856,0.153289,-0.283008,-0.555665,-0.439498,0.785101,-0.155914,-1.116482,-0.683583,0.08833,0.411546
-1.353472,0.404135,0.167922,-0.324673,-0.625215,-0.418999,0.672451,-0.158293,-1.059672,-0.66215,0.094859,0.398162
-1.41747,0.350694,0.197188,-0.408003,-0.764315,-0.377999,0.447151,-0.163052,-0.946051,-0.619285,0.107917,0.371395


In [259]:
df_pca.describe()

statistic,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",-1.4366e-16,1.2432e-16,-1.6576e-17,8.2878e-18,1.6576e-17,-1.1050e-17,6.6303e-17,-5.5252e-17,-7.7353e-17,-7.4590e-17,-8.9785e-18,-4.2820e-17
"""std""",1.671421,1.571462,1.326849,1.226593,1.109965,0.940957,0.877639,0.812326,0.727373,0.609722,0.478962,0.409636
"""min""",-3.155398,-3.056202,-2.522623,-2.921009,-3.084877,-1.296769,-3.894343,-1.608288,-1.596398,-1.087096,-0.975328,-1.093947
"""25%""",-1.251004,-0.987825,-0.68221,-0.715957,-0.713447,-0.517181,-0.591398,-0.530525,-0.449696,-0.461062,-0.292013,-0.349081
"""50%""",-0.086811,-0.038677,-0.081856,-0.107492,0.041456,-0.126058,0.120074,-0.081411,-0.056779,-0.068896,-0.018839,0.093058
"""75%""",0.655301,0.973602,0.509383,0.799596,0.654078,0.302718,0.593107,0.522618,0.291119,0.336484,0.23024,0.317859
"""max""",4.648756,3.535733,4.2696,3.574884,4.379859,4.380695,2.405554,1.521652,2.622772,1.750866,1.039316,0.698489


## Split testing and traiing data

In [260]:
y = pl.read_csv(f"{resources_path}/y.csv", schema=pl.Schema({"tempering_time_s_proc":pl.String,
                                                             "tempering_temperature_C_proc":pl.String}))

In [261]:
X_train, X_test, y_train, y_test = train_test_split(df_pca.to_numpy(), y.to_numpy(), random_state=random_state, stratify=y)

# Modelling

## Random ForestCclassifier Model

In [262]:
rf_classifier = RandomForestClassifier(n_estimators=10, 
                                      criterion='entropy',
                                      random_state=random_state)

rf_classifier.fit(X_train, y_train)

# rf_classifier = MultiOutputClassifier(RandomForestClassifier()).fit(X_train, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=10,
                       random_state=2112250415)

In [263]:
y_pred = rf_classifier.predict(X_test)
y_pred


array([['86400', '100'],
       ['3600', '700'],
       ['3600', '200'],
       ['600', '350'],
       ['600', '500'],
       ['3600', '400'],
       ['86400', '300'],
       ['115200', '200'],
       ['9000', '200'],
       ['600', '600'],
       ['9000', '300'],
       ['900', '600'],
       ['3600', '250'],
       ['7200', '400'],
       ['600', '350'],
       ['86400', '500'],
       ['86400', '200'],
       ['3600', '700'],
       ['14400', '700'],
       ['86400', '300'],
       ['90', '300'],
       ['3600', '400'],
       ['600', '450'],
       ['900', '300'],
       ['900', '550'],
       ['86400', '200'],
       ['3600', '200'],
       ['86400', '350'],
       ['900', '600'],
       ['14400', '600'],
       ['900', '500'],
       ['90', '100'],
       ['115200', '500'],
       ['600', '400'],
       ['86400', '700'],
       ['90', '300'],
       ['90', '300'],
       ['14400', '300'],
       ['28800', '500'],
       ['900', '400'],
       ['900', '400'],
       ['600', '500']

In [264]:
labels = []
target_prefixes = ["ttime_", "ttemp_"]
target_units = ['_min', '_C']

for i in range(y_test.shape[1]):
    prefixed = list(map(lambda x: f"{target_prefixes[i]}{x}{target_units[i]}", rf_classifier.classes_[i]))
    labels.append(prefixed)

labels

[['ttime_10_min',
  'ttime_115200_min',
  'ttime_120_min',
  'ttime_14400_min',
  'ttime_28800_min',
  'ttime_300_min',
  'ttime_3600_min',
  'ttime_40_min',
  'ttime_57600_min',
  'ttime_600_min',
  'ttime_7200_min',
  'ttime_86400_min',
  'ttime_90_min',
  'ttime_900_min',
  'ttime_9000_min'],
 ['ttemp_100_C',
  'ttemp_200_C',
  'ttemp_250_C',
  'ttemp_300_C',
  'ttemp_350_C',
  'ttemp_400_C',
  'ttemp_450_C',
  'ttemp_500_C',
  'ttemp_550_C',
  'ttemp_600_C',
  'ttemp_700_C']]

In [270]:
classification_reports = []
for i in range(y_test.shape[1]):
    cr = classification_report(y_test[:, i], y_pred[:, i],  output_dict=True)#target_names=labels[i],)
    classification_reports.append(cr)

In [284]:
# polars does not parse the dicts correctly
cr_rf_ttemp = pd.DataFrame(classification_reports[1]).transpose()
cr_rf_ttime = pd.DataFrame(classification_reports[0]).transpose()

pl_cr_rf_ttemp = pl.DataFrame(cr_rf_ttemp.reset_index())\
    .filter(pl.col('index').str.contains('avg|acc').not_())\
        .with_columns(pl.col('index').cast(pl.Int32))\
    .sort('index', descending=False)
pl_cr_rf_ttime = pl.DataFrame(cr_rf_ttime.reset_index())\
    .filter(pl.col('index').str.contains('avg|acc').not_())\
    .with_columns(pl.col('index').cast(pl.Int32))\
    .sort('index', descending=False)

# display 2 DF inline https://softhints.com/display-two-pandas-dataframes-side-by-side-jupyter-notebook/
cr_rf_ttemp_styler = cr_rf_ttemp.style.set_table_attributes("style='display:inline'").set_caption('Tempering Temperature')
cr_rf_ttime_styler = cr_rf_ttime.style.set_table_attributes("style='display:inline'").set_caption('Tempering Time')

space = "\xa0" * 10
display_html(cr_rf_ttemp_styler._repr_html_() + space + cr_rf_ttime_styler._repr_html_(), raw=True)

,precision,recall,f1-score,support
100,0.513514,0.633333,0.567164,30.000000
200,0.378378,0.400000,0.388889,35.000000
250,0.222222,0.133333,0.166667,30.000000
300,0.400000,0.444444,0.421053,36.000000
350,0.214286,0.214286,0.214286,14.000000
400,0.377778,0.425000,0.400000,40.000000
450,0.142857,0.133333,0.137931,15.000000
500,0.171429,0.150000,0.160000,40.000000
550,0.166667,0.166667,0.166667,18.000000
600,0.232558,0.263158,0.246914,38.000000


In [289]:

rf_cr_ttemp_supp_plot = alt.Chart(pl_cr_rf_ttemp)\
    .mark_bar().encode(
        alt.X('index:O').title('Tempering Temperature (C)'),
        alt.Y('support'),
        alt.Color('f1-score')
        )

rf_cr_ttime_supp_plot = alt.Chart(pl_cr_rf_ttime)\
    .mark_bar().encode(
        alt.X('index:O').title('Tempering Time (s)'),
        alt.Y('support'),
        alt.Color('f1-score')
        )
rf_cr_ttemp_supp_plot

alt.concat(rf_cr_ttemp_supp_plot, rf_cr_ttime_supp_plot)

alt.ConcatChart(...)

## Data is quite imbalanced

SMOTE? sampling? IMBlearn library synthetic minority oversampling technique

An advanced model will be used first. 

## Deep Neural Network Model